# Search in the INSEE database

The code here-in-below shows how to search in the csv file from INSEE after they have been formatted in Insee_formatting.ipynb.
Various methods to look for data are timed and compared

Dr. M. Fortin, Sept. 2020

### Importing libraries


In [1]:
import pandas as pd
import numpy as np

### List of formatted csv in the zip file obtained as explained in Insee_formatting.ipynb

In [2]:
from zipfile import ZipFile
zip_file = ZipFile('Insee.zip')
file_name=[text_file.filename for text_file in zip_file.infolist()]
print(file_name)       

['Insee_00s.csv', 'Insee_10s.csv', 'Insee_70s.csv', 'Insee_80s.csv', 'Insee_90s.csv']


# 1. French presidents

## 1.1 Georges Pompidou

In [3]:
nom_recherche="Pompidou"
prenom_recherche="Georges"
genre="1" # only male presidents so far :(
decade='70s'

In [4]:
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 
insee

,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
0,DUCRET,MARIE ANTOINETTE,2,09,01,1922,01004,AMBERIEU-EN-BUGEY,FRANCE,10,12,1970,01421,6
1,GRANGEON,ERIC JEAN REMY,1,29,03,1969,01004,AMBERIEU-EN-BUGEY,FRANCE,25,04,1970,69383,1059
2,VELLET,PHILIPPE,1,01,02,1970,01004,AMBERIEU-EN-BUGEY,FRANCE,03,02,1970,01004,12
3,PRESSAVIN,LYDIE,2,06,04,1970,01004,AMBERIEU-EN-BUGEY,FRANCE,06,04,1970,01004,33
4,DOUAT,MARIE-SYLVIA MARTINE,2,08,07,1970,01004,AMBERIEU-EN-BUGEY,FRANCE,08,07,1970,01053,457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329654,LOUBEAU,NADEGE CATHERINE PASCALE,2,12,06,1971,95680,VILLIERS-LE-BEL,FRANCE,04,02,1979,75114,421
3329655,DENEL,YASMINE ALEXANDRA,2,17,05,1977,95680,VILLIERS-LE-BEL,FRANCE,25,12,1979,60175,348
3329656,WEISSENSEEL,CELINE,2,20,02,1979,95680,VILLIERS-LE-BEL,FRANCE,22,02,1979,93048,164
3329657,GUSTAN,JEAN-LOUIS,1,09,05,1979,95680,VILLIERS-LE-BEL,FRANCE,05,09,1979,95500,390


#### Columns
1. nom: family name
2. prenom(s): first name(s)
3. sexe: gender 1 for male and 2 for female
4. journaiss: day of birth
5. monthnaiss: month of birth 
6. yearnaiss: year of birth
7. lieunaiss: postcode of the place of birth
8. commnaiss: place of birth
9. paysnaiss: country of birth
10. jourdeces: day of death
11. monthdeces: month of death
12. yeardeces: year of death
14. lieudeces: postcode of the place of death
15. actedeces: reference of the death record

### 1.1.1. Time the search for gender

#### Using where()

In [5]:
%%time
insee[insee.sexe.where(insee.sexe==genre).notnull()].head()

CPU times: user 602 ms, sys: 0 ns, total: 602 ms
Wall time: 601 ms


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
1,GRANGEON,ERIC JEAN REMY,1,29,03,1969,01004,AMBERIEU-EN-BUGEY,FRANCE,25,04,1970,69383,1059
2,VELLET,PHILIPPE,1,01,02,1970,01004,AMBERIEU-EN-BUGEY,FRANCE,03,02,1970,01004,12
5,ROSIER,FELIX,1,25,11,1891,01004,AMBERIEU-EN-BUGEY,FRANCE,14,11,1970,30012,15
6,BOUVEYRON,PIERRE,1,27,04,1900,01005,AMBERIEUX-EN-DOMBES,FRANCE,11,12,1970,69383,2094
8,GIVORD,JACQUES,1,12,08,1910,01026,BAGE-LE-CHATEL,FRANCE,24,11,1970,06088,4880


#### Using a list comprehension

In [6]:
%%time
insee[insee.sexe==genre].head()

CPU times: user 478 ms, sys: 63.8 ms, total: 541 ms
Wall time: 545 ms


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
1,GRANGEON,ERIC JEAN REMY,1,29,03,1969,01004,AMBERIEU-EN-BUGEY,FRANCE,25,04,1970,69383,1059
2,VELLET,PHILIPPE,1,01,02,1970,01004,AMBERIEU-EN-BUGEY,FRANCE,03,02,1970,01004,12
5,ROSIER,FELIX,1,25,11,1891,01004,AMBERIEU-EN-BUGEY,FRANCE,14,11,1970,30012,15
6,BOUVEYRON,PIERRE,1,27,04,1900,01005,AMBERIEUX-EN-DOMBES,FRANCE,11,12,1970,69383,2094
8,GIVORD,JACQUES,1,12,08,1910,01026,BAGE-LE-CHATEL,FRANCE,24,11,1970,06088,4880


#### Using Vectorization

In [7]:
%%time
f = np.vectorize(lambda haystack, needle: needle in haystack)
insee[f(insee.sexe, genre)].head()

CPU times: user 704 ms, sys: 56.5 ms, total: 761 ms
Wall time: 774 ms


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
1,GRANGEON,ERIC JEAN REMY,1,29,03,1969,01004,AMBERIEU-EN-BUGEY,FRANCE,25,04,1970,69383,1059
2,VELLET,PHILIPPE,1,01,02,1970,01004,AMBERIEU-EN-BUGEY,FRANCE,03,02,1970,01004,12
5,ROSIER,FELIX,1,25,11,1891,01004,AMBERIEU-EN-BUGEY,FRANCE,14,11,1970,30012,15
6,BOUVEYRON,PIERRE,1,27,04,1900,01005,AMBERIEUX-EN-DOMBES,FRANCE,11,12,1970,69383,2094
8,GIVORD,JACQUES,1,12,08,1910,01026,BAGE-LE-CHATEL,FRANCE,24,11,1970,06088,4880


#### Conclusion: The list comprehension appears to be the fastest

### 1.1.2 Time the search for the family name (nom)

#### Using a list comprehension

In [8]:
%%time
insee[(insee['nom'] == nom_recherche.upper())].head()

CPU times: user 237 ms, sys: 4.05 ms, total: 241 ms
Wall time: 239 ms


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
69210,POMPIDOU,FRANCOIS,1,28,11,1893,24049,BORN-DE-CHAMPS,FRANCE,22,04,1971,24497,1
458379,POMPIDOU,BENJAMIN,1,02,06,1930,75104,PARIS 4,FRANCE,11,08,1972,25523,5
677436,POMPIDOU,GABRIEL JEAN,1,21,04,1916,46038,BRETENOUX,FRANCE,08,01,1973,91174,24/1
677886,POMPIDOU,ANNE JUSTINE,2,07,03,1895,46117,GAGNAC-SUR-CERE,FRANCE,29,03,1973,46038,4
693150,POMPIDOU,HERVE BRUNO,1,18,04,1970,51454,REIMS,FRANCE,06,11,1973,51454,1965


#### Using find()

In [9]:
%%time
insee[ insee['nom'].str.find(nom_recherche.upper())==0].head()

CPU times: user 1.99 s, sys: 12.1 ms, total: 2.01 s
Wall time: 2 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
69210,POMPIDOU,FRANCOIS,1,28,11,1893,24049,BORN-DE-CHAMPS,FRANCE,22,04,1971,24497,1
458379,POMPIDOU,BENJAMIN,1,02,06,1930,75104,PARIS 4,FRANCE,11,08,1972,25523,5
677436,POMPIDOU,GABRIEL JEAN,1,21,04,1916,46038,BRETENOUX,FRANCE,08,01,1973,91174,24/1
677886,POMPIDOU,ANNE JUSTINE,2,07,03,1895,46117,GAGNAC-SUR-CERE,FRANCE,29,03,1973,46038,4
693150,POMPIDOU,HERVE BRUNO,1,18,04,1970,51454,REIMS,FRANCE,06,11,1973,51454,1965


#### Using match()

In [10]:
%%time
insee[insee.nom.str.match(nom_recherche.upper())==True].head()

CPU times: user 1.63 s, sys: 3.87 ms, total: 1.63 s
Wall time: 1.63 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
69210,POMPIDOU,FRANCOIS,1,28,11,1893,24049,BORN-DE-CHAMPS,FRANCE,22,04,1971,24497,1
458379,POMPIDOU,BENJAMIN,1,02,06,1930,75104,PARIS 4,FRANCE,11,08,1972,25523,5
677436,POMPIDOU,GABRIEL JEAN,1,21,04,1916,46038,BRETENOUX,FRANCE,08,01,1973,91174,24/1
677886,POMPIDOU,ANNE JUSTINE,2,07,03,1895,46117,GAGNAC-SUR-CERE,FRANCE,29,03,1973,46038,4
693150,POMPIDOU,HERVE BRUNO,1,18,04,1970,51454,REIMS,FRANCE,06,11,1973,51454,1965


#### Conclusion: again the list comprehension is the fastest

### 1.1.3. Check if adding the constraint on gender slows down the search

#### Case A:
1. gender
2. family name
3. first name

In [11]:
%%time
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 

insee=insee[(insee['nom'] == nom_recherche.upper())]
insee=insee[insee.prenoms.str.contains(prenom_recherche.upper())]
insee=insee[insee.sexe==genre]

insee

CPU times: user 11.2 s, sys: 640 ms, total: 11.9 s
Wall time: 11.9 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
933471,POMPIDOU,GEORGES JEAN RAYMOND,1,05,07,1911,15129,MONTBOUDIF,FRANCE,02,04,1974,75104,277


#### Case B:
1. family name
2. first name

In [12]:
%%time
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 

insee=insee[(insee['nom'] == nom_recherche.upper())]
insee=insee[insee.prenoms.str.contains(prenom_recherche.upper())]
insee

CPU times: user 11.4 s, sys: 604 ms, total: 12 s
Wall time: 12 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
933471,POMPIDOU,GEORGES JEAN RAYMOND,1,05,07,1911,15129,MONTBOUDIF,FRANCE,02,04,1974,75104,277


**Conclusion** comparing cases A and B: adding the gender seems to slow down the search a little

### 1.1.4. Compare the search order 

#### Case C:
1. first name
2. family name

In [13]:
%%time
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 

insee=insee[insee.prenoms.str.contains(prenom_recherche.upper(), na=False)]
insee=insee[(insee['nom'] == nom_recherche.upper())]
insee

CPU times: user 13.1 s, sys: 672 ms, total: 13.8 s
Wall time: 13.8 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
933471,POMPIDOU,GEORGES JEAN RAYMOND,1,05,07,1911,15129,MONTBOUDIF,FRANCE,02,04,1974,75104,277


**Conclusion** comparing cases B and C: family name then first name is faster (the search of the family name is more restrictive than the one of the first name obviously)

### 1.1.5. Check if one should combine the searches or do it one by one

#### Case D: first name & family name simultaneously

In [14]:
%%time
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 
  
insee[(insee['nom'] == nom_recherche.upper()) &(insee.prenoms.str.contains(prenom_recherche.upper()))]

CPU times: user 15.6 s, sys: 762 ms, total: 16.3 s
Wall time: 16.5 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
933471,POMPIDOU,GEORGES JEAN RAYMOND,1,05,07,1911,15129,MONTBOUDIF,FRANCE,02,04,1974,75104,277


#### Conclusion: comparing cases D and B: non-simultaneous searches are faster

### 1.1.6. Conclusions:
All in all it is faster to:
* Use list comprehension when possible
* Do searches one by one, starting with the most restrictive one here the family name

## 1.2. Francois Mitterand

In [15]:
nom_recherche="Mitterrand"
prenom_recherche="Francois"
genre="1"
decade='90s'

In [16]:
%%time
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 

insee=insee[(insee['nom'] == nom_recherche.upper())]
insee[insee.prenoms.str.contains(prenom_recherche.upper())]  

CPU times: user 20.7 s, sys: 1.1 s, total: 21.8 s
Wall time: 21.8 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
3386468,MITTERRAND,FRANCOIS MARIE ADRIEN MAURICE,1,26,10,1916,16167,JARNAC,FRANCE,08,01,1996,75107,16


## 1.3. Jacques Chirac

In [17]:
nom_recherche="Chirac"
prenom_recherche="Jacques"
genre="1"
decade='10s'

In [18]:
%%time
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 

insee=insee[(insee['nom'] == nom_recherche.upper())]
insee[insee.prenoms.str.contains(prenom_recherche.upper())]

CPU times: user 4.58 s, sys: 232 ms, total: 4.81 s
Wall time: 4.81 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
1114658,CHIRAC,JACQUES RENE,1,29,11,1932,75105,PARIS 5E ARRONDISSEMENT,FRANCE,26,09,2019,75106,129


# 2. Other personalities

## 2.1. Agnes Varda

I know that her first name is not necessarily the real one. Hence I use her name and gender for the search

In [19]:
nom_recherche="Varda"
genre="2"
decade='10s'

In [20]:
%%time 
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 

insee=insee[insee.sexe==genre]  
insee[(insee['nom'] == nom_recherche.upper())]

CPU times: user 4.63 s, sys: 196 ms, total: 4.83 s
Wall time: 4.83 s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
822313,VARDA,ARLETTE,2,30,05,1928,99131,IXELLES,BELGIQUE,29,03,2019,75114,670
873434,VARDA,HELENE,2,23,08,1924,75116,PARIS 16E ARRONDISSEMENT,FRANCE,28,04,2019,75115,1370


I know that Agnès Varda was born in Belgium so her civil name was Arlette Varda and she was on May 30, 1928 and passed on March 29, 2019.

## 2.2. Francois Truffaut

I do not know when he passed hence I am checking all files.
Also I want to find people whose first names start with Francois, hence the use of startwith().
Finally in French the female name Francoise starts similarly to male name Francois, hence I restrict to male, checking for gender.

In [21]:
nom_recherche="Truffaut"
prenom_recherche="Francois"
genre="1"

In [22]:
%%time
results = pd.DataFrame()
for filename in file_name:
        insee = pd.DataFrame()
        insee = pd.read_csv(zip_file.open(filename),dtype=str, index_col=0) 
        insee=insee[insee['nom'] == nom_recherche.upper()]
        insee=insee[insee.prenoms.str.startswith(prenom_recherche.upper(),na=False)]
        results = pd.concat([results,insee[insee.sexe==genre]])
results        

CPU times: user 1min 14s, sys: 3.96 s, total: 1min 18s
Wall time: 1min 18s


,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
4745581,TRUFFAUT,FRANCOIS,1,24,12,1920,59512,ROUBAIX,FRANCE,20,07,2008,71076,675
2206511,TRUFFAUT,FRANCOIS ROLAND,1,06,02,1932,75117,PARIS 17,FRANCE,21,10,1984,92051,555


Since I have no recollection of Truffaut passing in 2008 when I lived in Paris, I conclude that the screen director passed in 1984.

# 3. My paternal grandfather

I know his gender obviously and his family name and that he passed in the 70s.

In [23]:
genre="1"
nom_recherche="FORTIN"
decade='70s'

In [24]:
insee = pd.DataFrame()
insee=pd.read_csv(zip_file.open(file_name[file_name.index('Insee_'+decade+'.csv')]),dtype=str, index_col=0) 
insee=insee[(insee.nom == nom_recherche.upper())]
fortin=insee[(insee.sexe ==genre)]
fortin

,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
25072,FORTIN,GUY JEAN GILBERT,1,04,08,1948,83118,SAINT-RAPHAEL,FRANCE,06,05,1970,06088,2049
25492,FORTIN,GUY ALEXANDRE ROGER,1,05,07,1947,85122,LESSON,FRANCE,28,07,1970,36018,54
32451,FORTIN,ANDRE LOUIS,1,17,08,1927,03118,GANNAT,FRANCE,09,12,1971,03197,13
38316,FORTIN,ELISEE CHARLES JOSEPH,1,19,03,1902,08070,BLANZY-LA-SALONNAISE,FRANCE,29,12,1971,51105,18
48357,FORTIN,ANDRE GASTON RENE,1,10,06,1906,14102,LE BREUIL-EN-AUGE,FRANCE,08,12,1971,14366,527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283727,FORTIN,GABRIEL FELIX,1,26,01,1899,78350,LOUVECIENNES,FRANCE,27,01,1979,22325,6
3285664,FORTIN,ROLAND EDOUARD HYACINTHE,1,30,11,1936,78521,RIS-ORANGIS,FRANCE,25,11,1979,89024,792
3293279,FORTIN,ROBERT DESIRE JULES PAUL,1,17,01,1926,80024,ARGOEUVES,FRANCE,29,03,1979,80024,2
3308767,FORTIN,JOSEPH PIERRE LUCIEN,1,09,04,1941,85090,LA FLOCELLIERE,FRANCE,18,01,1979,79011,3


I also know he was born and died in the same departement (territorial subdivision) in Normandy so the citycode (lieunaiss/lieudeces) where he was born starts with 14, 61, 50, 76, 27.

In [25]:
results = pd.DataFrame()

departement=["14","61","50","76","27"]
for dep in departement:
    results=pd.concat([results,fortin[(fortin.lieunaiss.str.startswith(dep)) & (fortin.lieudeces.str.startswith(dep))]])
results

,nom,prenoms,sexe,journaiss,moisnaiss,anneenaiss,lieunaiss,commnaiss,paysnaiss,jourdeces,moisdeces,anneedeces,lieudeces,actedeces
48357,FORTIN,ANDRE GASTON RENE,1,10,06,1906,14102,LE BREUIL-EN-AUGE,FRANCE,08,12,1971,14366,527
1313988,FORTIN,BERNARD GEORGES LEOPOLD LEON,1,25,10,1901,14654,SAINT-PIERRE-SUR-DIVES,FRANCE,17,10,1975,14118,1588
1314539,FORTIN,RAYMOND CHARLES,1,11,08,1908,14762,VIRE,FRANCE,12,07,1975,14762,100
1714801,FORTIN,ALAIN LUCIEN GEORGES,1,13,05,1952,14762,VIRE,FRANCE,12,04,1976,14118,534
2120975,FORTIN,CHRISTIAN ALAIN CEDRIC,1,12,09,1977,14118,CAEN,FRANCE,05,11,1977,14118,1896
2529773,FORTIN,FRANCIS ANDRE VICTOR,1,17,11,1918,14464,NEUVILLE,FRANCE,02,11,1978,14118,1898
1512252,FORTIN,FERNAND PIERRE RAYMOND ANDRE,1,13,07,1914,61093,CHANU,FRANCE,07,02,1975,61093,4
1918625,FORTIN,GERARD RAYMOND MICHEL,1,12,01,1955,61415,SAINT-LEGER-SUR-SARTHE,FRANCE,20,06,1976,61001,191
2738546,FORTIN,ADOLPHE HAZAEL EDMOND,1,04,03,1908,61273,MENIL-VIN,FRANCE,16,01,1978,61169,23
2738596,FORTIN,AUGUSTE LEON,1,10,07,1899,61287,MONTILLY-SUR-NOIREAU,FRANCE,30,08,1978,61391,32


I also know his first names (prenoms) and the name of his birthplace (commnaiss) but I will keep that secret... All I can say is that my grandfather is indeed in the list just above :)